In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
# Carregar modelo treinado
model = load_model('mobilenetv2_emotion_finetuned_07-05.keras')

2025-05-07 22:02:30.724397: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-05-07 22:02:30.724546: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-05-07 22:02:30.724553: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
I0000 00:00:1746666150.724886  487829 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1746666150.724937  487829 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/opt/homebrew/Caskroom/miniforge/base/envs/tf-m1/lib/python3.12/site-packages/keras/src/trainers/trainer.py:212: UserWarning: Model doesn't support `jit_compile=True`. Proceeding with `jit_compile=False`.
  warnings.warn(


In [3]:
# Rótulos das classes (ajuste conforme seu treinamento)
class_labels = ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

In [4]:
# Inicializar detector Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [5]:
# Tamanho esperado pelo modelo
img_size = 96

In [ ]:
# Inicializar webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise RuntimeError("Não foi possível acessar a webcam.")

print("Pressione 'q' para sair.")

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_rgb = cv2.resize(face, (img_size, img_size))
        face_rgb = cv2.cvtColor(face_rgb, cv2.COLOR_BGR2RGB)
        face_input = np.expand_dims(face_rgb, axis=0) / 255.0

        prediction = model.predict(face_input, verbose=0)
        predicted_class = np.argmax(prediction)
        predicted_label = class_labels[predicted_class]

        # Desenhar bounding box e rótulo
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, predicted_label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('Deteccao de Emocoes - Webcam', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()

Pressione 'q' para sair.


2025-05-07 22:02:34.381145: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


: 